In [ ]:
from DobotEDU import *
from ultralytics import YOLO
import cv2
import numpy as np
import math
from playsound import playsound #bisa dilewatkan jika tidak ingin ada output suara saat skip giliran / giliran habis. pastikan di kode utama dihapus

# Define the minimum bounding box size threshold
min_bbox_size = 30  # Adjust this threshold as needed

def convert_coordinates(x, y): #Mengambil koordinat kamera (x, y) sebagai input dan mengonversinya ke koordinat dunia nyata
    baselines = [(214, 261), (214, 313), (352, 252), (331, 306), (426, 244), (430, 296), (389, 95), (399, 159), (308, 103), (314, 156), (211, 120), (214, 168)]
    distances = [np.sqrt((x - baseline[0])**2 + (y - baseline[1])**2) for baseline in baselines]
    closest_index = np.argmin(distances)
    
    if closest_index == 0:
        return -60.12, -200.78
    elif closest_index == 1:
        return -60.12, -200.78
    elif closest_index == 2:
        return -66.49, -248.62
    elif closest_index == 3:
        return -66.49, -248.62
    elif closest_index == 4:
        return -58.03, -301.88
    elif closest_index == 5:
        return -58.03, -301.88
    elif closest_index == 6:
        return 24.25, -301.88
    elif closest_index == 7:
        return 24.25, -301.88
    elif closest_index == 8:
        return 26.63, -253.8
    elif closest_index == 9:
        return 26.63, -253.8
    elif closest_index == 10:
        return 26.63, -202.6
    elif closest_index == 11:
        return 26.63, -202.6
    else:
        return None


def capture_card_robot(): #Mengambil gambar dari webcam dan mendeteksi kartu robot
    global detected_classes, x_coordinates, y_coordinates

    # Start webcam, CHANGE THIS using your own camera source. usually source 0 or use droidcam like our code. 
    droidcam_url = "http://10.3.133.106:4747/video"
    cap = cv2.VideoCapture(droidcam_url)

    if not cap.isOpened():
        print("Error: Webcam not found.")
        return

    # Model
    model = YOLO("best-2.pt")

    # First while loop for live feed
    while True:
        # Clear arrays before processing a new frame
        detected_classes = []
        x_coordinates = []
        y_coordinates = []

        # Capture live camera feed
        success, frame = cap.read()

        if not success:
            print("Error: Failed to capture frame.")
            break

        # Run YOLO model on the frame
        results = model(frame, stream=True)

        for r in results:
            boxes = r.boxes
            for box in boxes:
                # Bounding box
                x1, y1, x2, y2 = box.xyxy[0]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

                # Check bounding box size
                if abs(x2 - x1) < min_bbox_size or abs(y2 - y1) < min_bbox_size:
                    continue  # Skip small bounding boxes

                # Draw box on the frame
                cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 255), 3)

                # Calculate center of the bounding box
                center_x = int((x1 + x2) / 2)
                center_y = int((y1 + y2) / 2)

                # Draw blue dot at the center of the bounding box
                cv2.circle(frame, (center_x, center_y), 5, (255, 0, 0), -1)

                # Display coordinates of the center
                coord_text = f"({center_x}, {center_y})"
                cv2.putText(frame, coord_text, (center_x - 20, center_y - 25), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

                # Confidence
                confidence = math.ceil((box.conf[0] * 100)) / 100

                # Class name
                cls = int(box.cls[0])
                class_name = str(cls)

                # Display class name near the bounding box
                class_text = f"Class: {class_name}"
                cv2.putText(frame, class_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

                x_value, y_value = center_x, center_y

                # Append data to global arrays
                detected_classes.append(int(class_name))
                x_coordinates.append(float(x_value))
                y_coordinates.append(float(y_value))

        # Display detected objects with bounding boxes
        cv2.imshow("Detected Objects", frame)

        # Check for key press
        key = cv2.waitKey(1)
        if key == ord('q'):
            # Save the captured image
            cv2.imwrite("kartu_robotq.jpg", frame)

            # Close the webcam and live feed
            cap.release()
            cv2.destroyAllWindows()

            # Second while loop for displaying captured image
            while True:
                # Display captured image with detected objects
                captured_image = cv2.imread("kartu_robotq.jpg")
                for i in range(len(detected_classes)):
                    cv2.rectangle(captured_image, (int(x_coordinates[i]), int(y_coordinates[i])), (int(x_coordinates[i]) + 50, int(y_coordinates[i]) + 50), (255, 0, 0), -1)
                cv2.imshow("Captured Image with Objects", captured_image)

                key = cv2.waitKey(1)
                if key == ord('w'):
                    # Print the arrays
                    print("1. Array kartu robot dideteksi:", detected_classes)
                    print("2. Array koor x kartu robot:", x_coordinates)
                    print("3. Array koor y kartu robot:", y_coordinates)

                    cv2.destroyAllWindows()
                    return

def capture_card_meja(): #Mengambil gambar dari webcam dan mendeteksi kartu meja
    global kartu_meja, x_meja, y_meja

    # Start webcam
    droidcam_url = "http://10.3.133.63:4747/video"
    cap = cv2.VideoCapture(droidcam_url)

    if not cap.isOpened():
        print("Error: Webcam not found.")
        return

    # Model
    model = YOLO("best-2.pt")

    # First while loop for live feed
    while True:
        # Clear arrays before processing new frame
        kartu_meja = []
        x_meja = []
        y_meja = []

        # Capture live camera feed
        success, frame = cap.read()

        if not success:
            print("Error: Failed to capture frame.")
            break

        # Run YOLO model on the frame
        results = model(frame, stream=True)

        for r in results:
            boxes = r.boxes
            for box in boxes:
                # Bounding box
                x1, y1, x2, y2 = box.xyxy[0]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

                # Check bounding box size
                if abs(x2 - x1) < min_bbox_size or abs(y2 - y1) < min_bbox_size:
                    continue  # Skip small bounding boxes

                # Calculate center of the bounding box
                center_x = int((x1 + x2) / 2)
                center_y = int((y1 + y2) / 2)

                # Append data to global arrays
                kartu_meja.append(int(box.cls[0]))
                x_meja.append(center_x)
                y_meja.append(center_y)

                # Display detected class on the live feed
                class_text = f"Class: {int(box.cls[0])} ({math.ceil((box.conf[0] * 100)) / 100})"
                cv2.putText(frame, class_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

                # Draw box on the frame only if the class is in kartu_meja
                if int(box.cls[0]) in kartu_meja:
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 255), 3)

                    # Draw blue dot at the center of the bounding box
                    cv2.circle(frame, (center_x, center_y), 5, (255, 0, 0), -1)

                    # Display coordinates of the center
                    coord_text = f"({center_x}, {center_y})"
                    cv2.putText(frame, coord_text, (center_x - 20, center_y - 25), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                                (0, 255, 0), 1)

        # Display detected objects with bounding boxes in the live feed
        cv2.imshow("Live Camera Feed", frame)

        # Check for key press
        key = cv2.waitKey(1)
        if key == ord('q'):
            # Save the captured image
            cv2.imwrite("kartu_meja.jpg", frame)

            # Close the webcam and live feed
            cap.release()
            cv2.destroyAllWindows()

            # Second while loop for displaying captured image
            while True:
                # Display captured image with detected objects
                captured_image = cv2.imread("kartu_meja.jpg")
                for i in range(len(kartu_meja)):
                    # Draw bounding box
                    cv2.rectangle(captured_image, (int(x_meja[i]), int(y_meja[i])),
                                  (int(x_meja[i]) + 50, int(y_meja[i]) + 50), (255, 0, 0), -1)

                    # Display class information next to the bounding box
                    class_text = f"Class: {kartu_meja[i]}"
                    cv2.putText(captured_image, class_text, (int(x_meja[i]) + 60, int(y_meja[i]) + 25),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

                cv2.imshow("Captured Image with Objects", captured_image)

                key = cv2.waitKey(1)
                if key == ord('w'):
                    # Print the arrays
                    print("1. Array kartu meja dideteksi:", kartu_meja)

                    cv2.destroyAllWindows()
                    return

def pilih_kartu(): #Mengambil kartu robot yang sesuai dengan kartu meja
    ind_c = -1

    for i in range(len(detected_classes)):
        if detected_classes[i] in kartu_meja:
            ind_c = i
            break

    if ind_c != -1:
        result = convert_coordinates(x_coordinates[ind_c], y_coordinates[ind_c])
        magician.set_endeffector_suctioncup(enable=True, on=True)
        magician.ptp(mode=0, x=result[0], y=result[1], z=-81, r=0)
        magician.ptp(mode=0, x=259.24, y=0.08, z=-8.51, r=0)
        magician.set_endeffector_suctioncup(enable=False, on=False)
        playsound("output.mp3")
        print("x_res:", result[0])
        print("y_res:", result[1])
        print("x_awal:", x_coordinates[ind_c])
        print("y_awal:", y_coordinates[ind_c])
        print("ind_c:", ind_c)
        print("detected_classes:", detected_classes)
        time.sleep(10)
        playsound("habis.mp3")
        detected_classes[ind_c] = -1
    else:
        playsound("skip.mp3")
        playsound("your_turn.mp3")
        time.sleep(10)
        playsound("habis.mp3")


counter = 1

while counter != -1:
   capture_card_robot()
   capture_card_meja()
   pilih_kartu()
